In [1]:
from data_common.notebook import *
from data_common.pandas.df_extensions import la
from datetime import date
import rich

In [2]:
# get councils as of this date
council_date = date(2023, 4, 2)

df = la.get_la_df(as_of_date=council_date)[["local-authority-code", "x", "y", "region"]]

assert df["region"].isnull().any() == False

In [3]:
labeldf = df[["local-authority-code", "region"]].rename(columns={"region": "label"})
labeldf.to_csv(
    Path("data", "packages", "physical_distance", "la_labels.csv"), index=False
)
labeldf.head()

,local-authority-code,label
0,ABC,Northern Ireland
1,ABD,Scotland
2,ABE,Scotland
3,ADU,South East
4,AGB,Scotland


In [4]:
labels = pd.Series(labeldf["label"].unique())
descs = labels.apply(lambda x: f"Local authorities in {x}")
descdf = pd.DataFrame({"label": labels, "desc": descs})
descdf.to_csv(
    Path("data", "packages", "physical_distance", "label_desc.csv"), index=False
)
descdf.head()

,label,desc
0,Northern Ireland,Local authorities in Northern Ireland
1,Scotland,Local authorities in Scotland
2,South East,Local authorities in South East
3,Wales,Local authorities in Wales
4,East Midlands,Local authorities in East Midlands


In [5]:
distance = (
    df.drop(columns=["region"])
    .set_index("local-authority-code")
    .space.self_distance(normalize=True)
    .space.match_distance()
    .space.local_rankings()
    .sort_values(["local-authority-code_A", "distance"])
)
distance.to_csv(
    Path("data", "packages", "physical_distance", "distance_map.csv"), index=False
)
distance.head()

,local-authority-code_A,local-authority-code_B,distance,match,position
0,ABC,NMD,0.23,95.1,1.0
1,ABC,MUL,0.24,94.9,2.0
2,ABC,LBC,0.25,94.7,3.0
3,ABC,ANN,0.26,94.4,4.0
4,ABC,BFS,0.33,93.0,5.0
